In [1]:
import pandas as pd
from pandas import DataFrame as DF
import numpy as np

# 1.GroupBy Mechanics

split-apply-combine: describing group operations

Stage Processes:
- Split into groups based on one or more keys
- A function is applied to each group, producing a new value
- Finally, the results of all those function applications are combined into a result object

In [2]:
df = pd.DataFrame({'key1' : ['a', 'a', 'b', 'b', 'a'],
                   'key2' : ['one', 'two', 'one', 'two', 'one'],
                   'data1' : np.random.randn(5),
                   'data2' : np.random.randn(5)})
df

,key1,key2,data1,data2
0,a,one,1.345349,0.656199
1,a,two,1.322775,-0.511659
2,b,one,0.408445,-1.026004
3,b,two,0.755489,0.081905
4,a,one,-0.105168,0.987746


In [3]:
grouped = df['data1'].groupby(df['key1'])
grouped

In [4]:
grouped.mean()

key1
a    0.854319
b    0.581967
Name: data1, dtype: float64

The result index has the name 'key1' because the DataFrame column df['key1'] did.

In [5]:
means = df['data1'].groupby([df['key1'], df['key2']]).mean()
means

key1  key2
a     one     0.620090
      two     1.322775
b     one     0.408445
      two     0.755489
Name: data1, dtype: float64

In [6]:
means.unstack()

key2,one,two
key1,,
a,0.620090,1.322775
b,0.408445,0.755489


In [7]:
df

,key1,key2,data1,data2
0,a,one,1.345349,0.656199
1,a,two,1.322775,-0.511659
2,b,one,0.408445,-1.026004
3,b,two,0.755489,0.081905
4,a,one,-0.105168,0.987746


In this example, the group keys are all Series, though they could be any arrays of the
right length:

In [8]:
states = np.array(['Ohio', 'California', 'California', 'Ohio', 'Ohio'])
years = np.array([2005, 2005, 2006, 2005, 2006])
sy = df['data1'].groupby([states, years]).mean()
sy

California  2005    1.322775
            2006    0.408445
Ohio        2005    1.050419
            2006   -0.105168
Name: data1, dtype: float64

In [9]:
sy.index

MultiIndex([('California', 2005),
            ('California', 2006),
            (      'Ohio', 2005),
            (      'Ohio', 2006)],
           )

In [10]:
df.groupby('key1').mean()

,data1,data2
key1,,
a,0.854319,0.377428
b,0.581967,-0.472050


df['key2'] is not numeric data, it is said to be a nuisance column, which is therefore excluded from the result

In [11]:
df.groupby(['key1', 'key2']).mean()

data1     data2
key1 key2                    
a    one   0.620090  0.821972
     two   1.322775 -0.511659
b    one   0.408445 -1.026004
     two   0.755489  0.081905

In [12]:
df.groupby(['key1', 'key2']).size()

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

## Iterating Over Groups

In [13]:
df

,key1,key2,data1,data2
0,a,one,1.345349,0.656199
1,a,two,1.322775,-0.511659
2,b,one,0.408445,-1.026004
3,b,two,0.755489,0.081905
4,a,one,-0.105168,0.987746


In [14]:
df.groupby('key1').mean()

,data1,data2
key1,,
a,0.854319,0.377428
b,0.581967,-0.472050


In [15]:
df.groupby('key1').size()

key1
a    3
b    2
dtype: int64

In [16]:
for name, group in df.groupby('key1'):
    print(name)
    print(group,'\n\n')

a
  key1 key2     data1     data2
0    a  one  1.345349  0.656199
1    a  two  1.322775 -0.511659
4    a  one -0.105168  0.987746 


b
  key1 key2     data1     data2
2    b  one  0.408445 -1.026004
3    b  two  0.755489  0.081905 




In [17]:
for (k1, k2), group in df.groupby(['key1', 'key2']):
    print((k1, k2))
    print(group)

('a', 'one')
  key1 key2     data1     data2
0    a  one  1.345349  0.656199
4    a  one -0.105168  0.987746
('a', 'two')
  key1 key2     data1     data2
1    a  two  1.322775 -0.511659
('b', 'one')
  key1 key2     data1     data2
2    b  one  0.408445 -1.026004
('b', 'two')
  key1 key2     data1     data2
3    b  two  0.755489  0.081905


In [18]:
pieces = dict(list(df.groupby('key1')))
pieces

{'a':   key1 key2     data1     data2
 0    a  one  1.345349  0.656199
 1    a  two  1.322775 -0.511659
 4    a  one -0.105168  0.987746, 'b':   key1 key2     data1     data2
 2    b  one  0.408445 -1.026004
 3    b  two  0.755489  0.081905}

In [19]:
pieces['a']

,key1,key2,data1,data2
0,a,one,1.345349,0.656199
1,a,two,1.322775,-0.511659
4,a,one,-0.105168,0.987746


In [20]:
## gropuing by dtype
df.dtypes

key1      object
key2      object
data1    float64
data2    float64
dtype: object

In [21]:
grouped = df.groupby(df.dtypes, axis=1)
for dtype, group in grouped:
    print(dtype)
    print(group)

float64
      data1     data2
0  1.345349  0.656199
1  1.322775 -0.511659
2  0.408445 -1.026004
3  0.755489  0.081905
4 -0.105168  0.987746
object
  key1 key2
0    a  one
1    a  two
2    b  one
3    b  two
4    a  one


## Selecting a Column or Subset of Columns

In [22]:
df.groupby('key1')['data1']
df.groupby('key1')[['data2']]
#are syntatic sugar for:
df['data1'].groupby(df['key1'])
df[['data2']].groupby(df['key1'])

To compute means for just the data2 column and get the result as a DataFrame

In [23]:
df.groupby(['key1', 'key2'])[['data2']].mean()

data2
key1 key2          
a    one   0.821972
     two  -0.511659
b    one  -1.026004
     two   0.081905

In [24]:
s_grouped = df.groupby(['key1', 'key2'])['data2']
s_grouped

In [25]:
s_grouped.mean()

key1  key2
a     one     0.821972
      two    -0.511659
b     one    -1.026004
      two     0.081905
Name: data2, dtype: float64

## Grouping with Dicts and Series

Grouping information may exist in a form other than an array. Ex: DataFrame

In [26]:
people = pd.DataFrame(np.random.randn(5, 5),
                      columns=['a', 'b', 'c', 'd', 'e'],
                      index=['Joe', 'Steve', 'Wes', 'Jim', 'Travis'])
people.iloc[2:3, [1, 2]] = np.nan
people

,a,b,c,d,e
Joe,-0.287132,-1.515741,-0.838919,-0.950812,-1.529777
Steve,0.099802,-0.371220,0.110799,0.356289,1.089410
Wes,-0.008206,NaN,NaN,-1.423755,-2.061432
Jim,1.669626,-2.978642,1.277151,0.964747,-0.197850
Travis,-0.375547,-1.493311,1.889622,0.536323,-0.978529


In [27]:
mapping = {'a': 'red', 'b': 'red', 'c': 'blue',
           'd': 'blue', 'e': 'red', 'f' : 'orange'}

by_column = people.groupby(mapping, axis=1)
by_column.sum()

,blue,red
Joe,-1.789731,-3.332650
Steve,0.467088,0.817992
Wes,-1.423755,-2.069638
Jim,2.241898,-1.506866
Travis,2.425944,-2.847387


In [28]:
map_series = pd.Series(mapping)
map_series

a       red
b       red
c      blue
d      blue
e       red
f    orange
dtype: object

In [29]:
people.groupby(map_series, axis=1).count()

,blue,red
Joe,2,3
Steve,2,3
Wes,1,2
Jim,2,3
Travis,2,3


## Grouping with Functions

Suppose you wanted to group by the length of the names;
while you could compute an array of string lengths, it’s simpler to just pass the len
function:

In [30]:
people.groupby(len).sum()

,a,b,c,d,e
3,1.374288,-4.494383,0.438232,-1.409820,-3.789059
5,0.099802,-0.371220,0.110799,0.356289,1.089410
6,-0.375547,-1.493311,1.889622,0.536323,-0.978529


Mixing functions with arrays, dicts, or Series is not a problem as everything gets con‐
verted to arrays internally:

In [31]:
key_list = ['one', 'one', 'one', 'two', 'two']
people.groupby([len, key_list]).min()

a         b         c         d         e
3 one -0.287132 -1.515741 -0.838919 -1.423755 -2.061432
  two  1.669626 -2.978642  1.277151  0.964747 -0.197850
5 one  0.099802 -0.371220  0.110799  0.356289  1.089410
6 two -0.375547 -1.493311  1.889622  0.536323 -0.978529

In [32]:
people = pd.DataFrame(np.random.randn(6, 6),
                      columns=['a', 'b', 'c', 'd', 'e', 'f'],
                      index=['Joe', 'Steve', 'Wes', 'Jim', 'Travis','Petra'])
people.iloc[2:3, [1, 2]] = np.nan



In [33]:
idx_srt = sorted(list(people.index),key=len)
idx_srt

['Joe', 'Wes', 'Jim', 'Steve', 'Petra', 'Travis']

In [34]:
people.index= idx_srt
people

,a,b,c,d,e,f
Joe,0.290049,0.581785,0.018538,-0.864471,0.767466,-0.773322
Wes,-0.928415,0.377399,-0.412371,1.297104,-0.885630,1.337529
Jim,-1.474650,NaN,NaN,-1.293956,-0.240160,-0.921652
Steve,-0.542809,-1.977428,-0.379630,0.599075,-1.214431,-0.620382
Petra,1.077553,0.832213,-0.121725,0.325754,1.563929,1.154425
Travis,1.924651,-0.353984,0.415755,0.999273,0.546091,-1.246305


In [35]:
key_list = ['one', 'one', 'two', 'one', 'one', 'one']
people.groupby([len, key_list]).sum()

a         b         c         d         e         f
3 one -0.638366  0.959185 -0.393833  0.432634 -0.118164  0.564207
  two -1.474650  0.000000  0.000000 -1.293956 -0.240160 -0.921652
5 one  0.534743 -1.145215 -0.501355  0.924829  0.349498  0.534043
6 one  1.924651 -0.353984  0.415755  0.999273  0.546091 -1.246305

### Grouping by Index Levels

In [36]:
columns = pd.MultiIndex.from_arrays([['US', 'US', 'US', 'JP', 'JP'],
                                     [1, 3, 5, 1, 3]],
                                    names=['cty', 'tenor'])
columns

MultiIndex([('US', 1),
            ('US', 3),
            ('US', 5),
            ('JP', 1),
            ('JP', 3)],
           names=['cty', 'tenor'])

In [37]:
hier_df = pd.DataFrame(np.random.randn(4, 5), columns=columns)
hier_df

cty          US                            JP          
tenor         1         3         5         1         3
0     -0.373152  0.451009  0.147845  1.415406  1.280396
1     -1.335835  0.472026  1.168905 -1.329416 -1.079675
2      0.674969 -0.254269 -0.338255 -1.168750  0.555374
3      2.062111 -0.381480  0.579542 -0.085415  0.565376

In [38]:
hier_df.groupby(level='cty', axis=1).count()    

cty,JP,US
0,2,3
1,2,3
2,2,3
3,2,3


# 2.Data Aggregation

In [39]:
df

,key1,key2,data1,data2
0,a,one,1.345349,0.656199
1,a,two,1.322775,-0.511659
2,b,one,0.408445,-1.026004
3,b,two,0.755489,0.081905
4,a,one,-0.105168,0.987746


In [40]:
grouped = df.groupby('key1')

In [41]:
grouped['data1'].quantile(0.9)

key1
a    1.340834
b    0.720785
Name: data1, dtype: float64

In [42]:
def peak_to_peak(arr):
    return arr.max() - arr.min()

In [43]:
grouped.agg(peak_to_peak)

,data1,data2
key1,,
a,1.450516,1.499405
b,0.347045,1.107909


In [44]:
grouped.describe()

data1                                                              \
     count      mean       std       min       25%       50%       75%   
key1                                                                     
a      3.0  0.854319  0.831016 -0.105168  0.608804  1.322775  1.334062   
b      2.0  0.581967  0.245398  0.408445  0.495206  0.581967  0.668728   

               data2                                                    \
           max count      mean       std       min       25%       50%   
key1                                                                     
a     1.345349   3.0  0.377428  0.787616 -0.511659  0.072270  0.656199   
b     0.755489   2.0 -0.472050  0.783410 -1.026004 -0.749027 -0.472050   

                          
           75%       max  
key1                      
a     0.821972  0.987746  
b    -0.195072  0.081905

Custom aggregation functions are generally much slower than the
optimized functions

## Column-Wise and Multiple Function Application

In [45]:
tips = pd.read_csv('tips.csv')
tips.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [46]:
tips['tip_pct'] = tips['tip'] / tips['total_bill']
tips[:6]   

,total_bill,tip,sex,smoker,day,time,size,tip_pct
0,16.99,1.01,Female,No,Sun,Dinner,2,0.059447
1,10.34,1.66,Male,No,Sun,Dinner,3,0.160542
2,21.01,3.50,Male,No,Sun,Dinner,3,0.166587
3,23.68,3.31,Male,No,Sun,Dinner,2,0.139780
4,24.59,3.61,Female,No,Sun,Dinner,4,0.146808
5,25.29,4.71,Male,No,Sun,Dinner,4,0.186240


In [47]:
tips.index

RangeIndex(start=0, stop=244, step=1)

In [48]:
grouped = tips.groupby(['day', 'smoker'])
grouped_pct = grouped['tip_pct']
grouped_pct.agg('mean')

day   smoker
Fri   No        0.151650
      Yes       0.174783
Sat   No        0.158048
      Yes       0.147906
Sun   No        0.160113
      Yes       0.187250
Thur  No        0.160298
      Yes       0.163863
Name: tip_pct, dtype: float64

In [49]:
grouped_pct.agg(['mean', 'std', peak_to_peak])

mean       std  peak_to_peak
day  smoker                                  
Fri  No      0.151650  0.028123      0.067349
     Yes     0.174783  0.051293      0.159925
Sat  No      0.158048  0.039767      0.235193
     Yes     0.147906  0.061375      0.290095
Sun  No      0.160113  0.042347      0.193226
     Yes     0.187250  0.154134      0.644685
Thur No      0.160298  0.038774      0.193350
     Yes     0.163863  0.039389      0.151240

If you pass a list of (name, function) tuples, the first element of each tuple will be used as
the DataFrame column names

In [50]:
grouped_pct.agg([('foo', 'mean'), ('bar', np.std)])

foo       bar
day  smoker                    
Fri  No      0.151650  0.028123
     Yes     0.174783  0.051293
Sat  No      0.158048  0.039767
     Yes     0.147906  0.061375
Sun  No      0.160113  0.042347
     Yes     0.187250  0.154134
Thur No      0.160298  0.038774
     Yes     0.163863  0.039389

With a DataFrame you have more options, as you can specify a list of functions to
apply to all of the columns or different functions per column.

In [51]:
functions = ['count', 'mean', 'max']
result = grouped['tip_pct', 'total_bill'].agg(functions)
result

tip_pct                     total_bill                  
              count      mean       max      count       mean    max
day  smoker                                                         
Fri  No           4  0.151650  0.187735          4  18.420000  22.75
     Yes         15  0.174783  0.263480         15  16.813333  40.17
Sat  No          45  0.158048  0.291990         45  19.661778  48.33
     Yes         42  0.147906  0.325733         42  21.276667  50.81
Sun  No          57  0.160113  0.252672         57  20.506667  48.17
     Yes         19  0.187250  0.710345         19  24.120000  45.35
Thur No          45  0.160298  0.266312         45  17.113111  41.19
     Yes         17  0.163863  0.241255         17  19.190588  43.11

As you can see, the resulting DataFrame has hierarchical columns, the same as you
would get aggregating each column separately and using concat to glue the results
together using the column names as the keys argument:

In [52]:
result['tip_pct']

count      mean       max
day  smoker                           
Fri  No          4  0.151650  0.187735
     Yes        15  0.174783  0.263480
Sat  No         45  0.158048  0.291990
     Yes        42  0.147906  0.325733
Sun  No         57  0.160113  0.252672
     Yes        19  0.187250  0.710345
Thur No         45  0.160298  0.266312
     Yes        17  0.163863  0.241255

As before, a list of tuples with custom names can be passed:

In [53]:
ftuples = [('Durchschnitt', 'mean'), ('Abweichung', np.var)]
grouped['tip_pct', 'total_bill'].agg(ftuples)


tip_pct              total_bill            
            Durchschnitt Abweichung Durchschnitt  Abweichung
day  smoker                                                 
Fri  No         0.151650   0.000791    18.420000   25.596333
     Yes        0.174783   0.002631    16.813333   82.562438
Sat  No         0.158048   0.001581    19.661778   79.908965
     Yes        0.147906   0.003767    21.276667  101.387535
Sun  No         0.160113   0.001793    20.506667   66.099980
     Yes        0.187250   0.023757    24.120000  109.046044
Thur No         0.160298   0.001503    17.113111   59.625081
     Yes        0.163863   0.001551    19.190588   69.808518

Now, suppose you wanted to apply potentially different functions to one or more of
the columns. 
(mapping of column names
to any of the function specifications listed so far:)

In [54]:
grouped.agg({'tip' : np.max, 'size' : 'sum'})

tip  size
day  smoker             
Fri  No       3.50     9
     Yes      4.73    31
Sat  No       9.00   115
     Yes     10.00   104
Sun  No       6.00   167
     Yes      6.50    49
Thur No       6.70   112
     Yes      5.00    40

In [55]:
grouped.agg({'tip_pct': ['min', 'max', 'mean', 'std'], 'size': 'sum'})

tip_pct                               size
                  min       max      mean       std  sum
day  smoker                                             
Fri  No      0.120385  0.187735  0.151650  0.028123    9
     Yes     0.103555  0.263480  0.174783  0.051293   31
Sat  No      0.056797  0.291990  0.158048  0.039767  115
     Yes     0.035638  0.325733  0.147906  0.061375  104
Sun  No      0.059447  0.252672  0.160113  0.042347  167
     Yes     0.065660  0.710345  0.187250  0.154134   49
Thur No      0.072961  0.266312  0.160298  0.038774  112
     Yes     0.090014  0.241255  0.163863  0.039389   40

In [56]:
for name, group in grouped:
    print(name)
    print(group,'\n\n')

('Fri', 'No')
     total_bill   tip     sex smoker  day    time  size   tip_pct
91        22.49  3.50    Male     No  Fri  Dinner     2  0.155625
94        22.75  3.25  Female     No  Fri  Dinner     2  0.142857
99        12.46  1.50    Male     No  Fri  Dinner     2  0.120385
223       15.98  3.00  Female     No  Fri   Lunch     3  0.187735 


('Fri', 'Yes')
     total_bill   tip     sex smoker  day    time  size   tip_pct
90        28.97  3.00    Male    Yes  Fri  Dinner     2  0.103555
92         5.75  1.00  Female    Yes  Fri  Dinner     2  0.173913
93        16.32  4.30  Female    Yes  Fri  Dinner     2  0.263480
95        40.17  4.73    Male    Yes  Fri  Dinner     4  0.117750
96        27.28  4.00    Male    Yes  Fri  Dinner     2  0.146628
97        12.03  1.50    Male    Yes  Fri  Dinner     2  0.124688
98        21.01  3.00    Male    Yes  Fri  Dinner     2  0.142789
100       11.35  2.50  Female    Yes  Fri  Dinner     2  0.220264
101       15.38  3.00  Female    Yes  Fri  D

### Returning Aggregated Data Without Row Indexes

In [57]:
tips.groupby(['day', 'smoker'], as_index=False).mean()

,day,smoker,total_bill,tip,size,tip_pct
0,Fri,No,18.420000,2.812500,2.250000,0.151650
1,Fri,Yes,16.813333,2.714000,2.066667,0.174783
2,Sat,No,19.661778,3.102889,2.555556,0.158048
3,Sat,Yes,21.276667,2.875476,2.476190,0.147906
4,Sun,No,20.506667,3.167895,2.929825,0.160113
5,Sun,Yes,24.120000,3.516842,2.578947,0.187250
6,Thur,No,17.113111,2.673778,2.488889,0.160298
7,Thur,Yes,19.190588,3.030000,2.352941,0.163863


# 3.Apply: General split-apply-combine

Suppose you wanted to select the top five tip_pct values by group. First, write a function that selects the rows with the largest values in a particular column:

In [58]:
def top(df, n=5, column='tip_pct'):
    return df.sort_values(by=column)[-n:]

top(tips, n=6)

,total_bill,tip,sex,smoker,day,time,size,tip_pct
109,14.31,4.00,Female,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Male,Yes,Sun,Dinner,4,0.280535
232,11.61,3.39,Male,No,Sat,Dinner,2,0.291990
67,3.07,1.00,Female,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Female,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Male,Yes,Sun,Dinner,2,0.710345


In [59]:
tips.sort_values(by='tip_pct').tail(6)

,total_bill,tip,sex,smoker,day,time,size,tip_pct
109,14.31,4.00,Female,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Male,Yes,Sun,Dinner,4,0.280535
232,11.61,3.39,Male,No,Sat,Dinner,2,0.291990
67,3.07,1.00,Female,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Female,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Male,Yes,Sun,Dinner,2,0.710345


In [60]:
tips.groupby('smoker').apply(top)

total_bill   tip     sex smoker   day    time  size   tip_pct
smoker                                                                   
No     88        24.71  5.85    Male     No  Thur   Lunch     2  0.236746
       185       20.69  5.00    Male     No   Sun  Dinner     5  0.241663
       51        10.29  2.60  Female     No   Sun  Dinner     2  0.252672
       149        7.51  2.00    Male     No  Thur   Lunch     2  0.266312
       232       11.61  3.39    Male     No   Sat  Dinner     2  0.291990
Yes    109       14.31  4.00  Female    Yes   Sat  Dinner     2  0.279525
       183       23.17  6.50    Male    Yes   Sun  Dinner     4  0.280535
       67         3.07  1.00  Female    Yes   Sat  Dinner     1  0.325733
       178        9.60  4.00  Female    Yes   Sun  Dinner     2  0.416667
       172        7.25  5.15    Male    Yes   Sun  Dinner     2  0.710345

What has happened here? The top function is called on each row group from the
DataFrame, and then the results are glued together using pandas.concat , labeling the
pieces with the group names. The result therefore has a hierarchical index whose
inner level contains index values from the original DataFrame.

In [61]:
tips.groupby('smoker').agg('mean')

,total_bill,tip,size,tip_pct
smoker,,,,
No,19.188278,2.991854,2.668874,0.159328
Yes,20.756344,3.008710,2.408602,0.163196


If you pass a function to apply that takes other arguments or keywords, you can pass
these after the function:

In [62]:
tips.groupby(['smoker', 'day']).apply(top, n=1, column='total_bill')

total_bill    tip     sex smoker   day    time  size  \
smoker day                                                              
No     Fri  94        22.75   3.25  Female     No   Fri  Dinner     2   
       Sat  212       48.33   9.00    Male     No   Sat  Dinner     4   
       Sun  156       48.17   5.00    Male     No   Sun  Dinner     6   
       Thur 142       41.19   5.00    Male     No  Thur   Lunch     5   
Yes    Fri  95        40.17   4.73    Male    Yes   Fri  Dinner     4   
       Sat  170       50.81  10.00    Male    Yes   Sat  Dinner     3   
       Sun  182       45.35   3.50    Male    Yes   Sun  Dinner     3   
       Thur 197       43.11   5.00  Female    Yes  Thur   Lunch     4   

                  tip_pct  
smoker day                 
No     Fri  94   0.142857  
       Sat  212  0.186220  
       Sun  156  0.103799  
       Thur 142  0.121389  
Yes    Fri  95   0.117750  
       Sat  170  0.196812  
       Sun  182  0.077178  
       Thur 197  0.115982

In [63]:
result = tips.groupby('smoker')['tip_pct'].describe()
result

,count,mean,std,min,25%,50%,75%,max
smoker,,,,,,,,
No,151.0,0.159328,0.039910,0.056797,0.136906,0.155625,0.185014,0.291990
Yes,93.0,0.163196,0.085119,0.035638,0.106771,0.153846,0.195059,0.710345


In [64]:
result.unstack('smoker')

       smoker
count  No        151.000000
       Yes        93.000000
mean   No          0.159328
       Yes         0.163196
std    No          0.039910
       Yes         0.085119
min    No          0.056797
       Yes         0.035638
25%    No          0.136906
       Yes         0.106771
50%    No          0.155625
       Yes         0.153846
75%    No          0.185014
       Yes         0.195059
max    No          0.291990
       Yes         0.710345
dtype: float64

In [65]:
result.T

smoker,No,Yes
count,151.000000,93.000000
mean,0.159328,0.163196
std,0.039910,0.085119
min,0.056797,0.035638
25%,0.136906,0.106771
50%,0.155625,0.153846
75%,0.185014,0.195059
max,0.291990,0.710345


In [66]:
f = lambda x: x.describe()
grouped.apply(f)

total_bill       tip  size   tip_pct
day  smoker                                            
Fri  No     count    4.000000  4.000000  4.00  4.000000
            mean    18.420000  2.812500  2.25  0.151650
            std      5.059282  0.898494  0.50  0.028123
            min     12.460000  1.500000  2.00  0.120385
            25%     15.100000  2.625000  2.00  0.137239
...                       ...       ...   ...       ...
Thur Yes    min     10.340000  2.000000  2.00  0.090014
            25%     13.510000  2.000000  2.00  0.148038
            50%     16.470000  2.560000  2.00  0.153846
            75%     19.810000  4.000000  2.00  0.194837
            max     43.110000  5.000000  4.00  0.241255

[64 rows x 4 columns]

## Suppressing the Group Keys

In [67]:
tips.groupby('smoker', group_keys=False).apply(top)

,total_bill,tip,sex,smoker,day,time,size,tip_pct
88,24.71,5.85,Male,No,Thur,Lunch,2,0.236746
185,20.69,5.00,Male,No,Sun,Dinner,5,0.241663
51,10.29,2.60,Female,No,Sun,Dinner,2,0.252672
149,7.51,2.00,Male,No,Thur,Lunch,2,0.266312
232,11.61,3.39,Male,No,Sat,Dinner,2,0.291990
109,14.31,4.00,Female,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Male,Yes,Sun,Dinner,4,0.280535
67,3.07,1.00,Female,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Female,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Male,Yes,Sun,Dinner,2,0.710345


## Quantile and Bucket Analysis

In [68]:
frame = pd.DataFrame({'data1': np.random.randn(1000),
                      'data2': np.random.randn(1000)})

quantiles = pd.cut(frame.data1, 4)
quantiles[:10]

0     (-0.113, 1.551]
1    (-1.777, -0.113]
2     (-0.113, 1.551]
3    (-1.777, -0.113]
4     (-0.113, 1.551]
5    (-1.777, -0.113]
6      (1.551, 3.215]
7    (-1.777, -0.113]
8     (-0.113, 1.551]
9    (-1.777, -0.113]
Name: data1, dtype: category
Categories (4, interval[float64]): [(-3.448, -1.777] < (-1.777, -0.113] < (-0.113, 1.551] < (1.551, 3.215]]

In [69]:
def get_stats(group):
    return{'min': group.min(), 'max': group.max(), 'count': group.count(), 'mean': group.mean()}

grouped = frame.data2.groupby(quantiles)
grouped.apply(get_stats).unstack()

,min,max,count,mean
data1,,,,
"(-3.448, -1.777]",-1.794222,1.648538,42.0,-0.306157
"(-1.777, -0.113]",-2.965164,3.371742,400.0,-0.050677
"(-0.113, 1.551]",-2.418057,2.738053,491.0,0.028072
"(1.551, 3.215]",-2.421590,2.059810,67.0,0.075368


## Example: Filling Missing Values with Group-Specific Values

In [70]:
s = pd.Series(np.random.randn(6))
s[::2] = np.nan
s

0         NaN
1   -1.108511
2         NaN
3   -0.430480
4         NaN
5   -0.361339
dtype: float64

In [71]:
s.fillna(s.mean())

0   -0.633444
1   -1.108511
2   -0.633444
3   -0.430480
4   -0.633444
5   -0.361339
dtype: float64

In [72]:
states = ['Ohio', 'New York', 'Vermont', 'Florida',
          'Oregon', 'Nevada', 'California', 'Idaho']

In [73]:
group_key = ['East'] * 4 + ['West'] * 4

In [74]:
data = pd.Series(np.random.randn(8), index=states)
data

Ohio         -2.143761
New York     -1.695443
Vermont       0.240354
Florida      -0.041708
Oregon       -0.911525
Nevada       -0.357958
California   -0.071783
Idaho        -0.212998
dtype: float64

In [75]:
data[['Vermont', 'Nevada', 'Idaho']] = np.nan
data

Ohio         -2.143761
New York     -1.695443
Vermont            NaN
Florida      -0.041708
Oregon       -0.911525
Nevada             NaN
California   -0.071783
Idaho              NaN
dtype: float64

We can fill the NA values using the group means like so:

In [76]:
data.groupby(group_key).agg('mean')

East   -1.293637
West   -0.491654
dtype: float64

In [77]:
fill_mean = lambda g: g.fillna(g.mean())
data.groupby(group_key).apply(fill_mean)

Ohio         -2.143761
New York     -1.695443
Vermont      -1.293637
Florida      -0.041708
Oregon       -0.911525
Nevada       -0.491654
California   -0.071783
Idaho        -0.491654
dtype: float64

In another case, you might have predefined fill values in your code that vary by
group. Since the groups have a name attribute set internally, we can use that:

In [78]:
fill_values = {'East': 0.5, 'West': -1}
fill_func = lambda g: g.fillna(fill_values[g.name])
data.groupby(group_key).apply(fill_func)

Ohio         -2.143761
New York     -1.695443
Vermont       0.500000
Florida      -0.041708
Oregon       -0.911525
Nevada       -1.000000
California   -0.071783
Idaho        -1.000000
dtype: float64

## Example: Random Sampling and Permutation

Construct a deck of English-style playing cards:

In [79]:
# Hearts, Spades, Clubs, Diamonds
suits = ['H', 'S', 'C', 'D']
card_val = (list(range(1, 11)) + [10] * 3) * 4
base_names = ['A'] + list(range(2, 11)) + ['J', 'K', 'Q']
cards = []
for suit in ['H', 'S', 'C', 'D']:
    cards.extend(str(num) + suit for num in base_names)
    
    
deck = pd.Series(card_val, index=cards)
print(len(deck))
deck[:13]

52


AH      1
2H      2
3H      3
4H      4
5H      5
6H      6
7H      7
8H      8
9H      9
10H    10
JH     10
KH     10
QH     10
dtype: int64

So now we have a Series of length 52 whose index contains card names and values are
the ones used in Blackjack and other games (to keep things simple, I just let the ace
'A' be 1):

In [80]:
def draw(deck, n=5):
    return deck.sample(n)

Suppose you wanted two random cards from each suit. Because the suit is the last
character of each card name, we can group based on this and use apply :

In [88]:
get_suit = lambda card: card[-1] #last letter is suit
for i,j in deck.groupby(get_suit):
    print(i,'\n\n',j,'\n\n')

C 

 AC      1
2C      2
3C      3
4C      4
5C      5
6C      6
7C      7
8C      8
9C      9
10C    10
JC     10
KC     10
QC     10
dtype: int64 


D 

 AD      1
2D      2
3D      3
4D      4
5D      5
6D      6
7D      7
8D      8
9D      9
10D    10
JD     10
KD     10
QD     10
dtype: int64 


H 

 AH      1
2H      2
3H      3
4H      4
5H      5
6H      6
7H      7
8H      8
9H      9
10H    10
JH     10
KH     10
QH     10
dtype: int64 


S 

 AS      1
2S      2
3S      3
4S      4
5S      5
6S      6
7S      7
8S      8
9S      9
10S    10
JS     10
KS     10
QS     10
dtype: int64 




In [89]:
deck.groupby(get_suit).apply(draw, n=2)


C  6C      6
   JC     10
D  5D      5
   10D    10
H  10H    10
   AH      1
S  5S      5
   KS     10
dtype: int64

In [99]:
##alternatively
rs = deck.groupby(get_suit, group_keys=False).apply(draw, n=2)
rs

KC    10
JC    10
7D     7
JD    10
KH    10
3H     3
6S     6
JS    10
dtype: int64

In [101]:
rs.index

Index(['KC', 'JC', '7D', 'JD', 'KH', '3H', '6S', 'JS'], dtype='object')

In [103]:
rs = rs.reset_index()

In [109]:
idx = np.random.permutation(rs.index)
idx

array([3, 1, 4, 2, 6, 7, 0, 5])

In [110]:
rs.reindex(index=idx)

,index,0
3,JD,10
1,JC,10
4,KH,10
2,7D,7
6,6S,6
7,JS,10
0,KC,10
5,3H,3


## Example: Group Weighted Average and Correlation

In [111]:
df = pd.DataFrame({'category': ['a', 'a', 'a', 'a',
                                'b', 'b', 'b', 'b'],
                   'data': np.random.randn(8),
                   'weights': np.random.rand(8)})
df

,category,data,weights
0,a,0.705889,0.132191
1,a,0.591469,0.927164
2,a,0.043058,0.090516
3,a,-0.459714,0.391590
4,b,-0.049132,0.368965
5,b,-0.393083,0.518976
6,b,0.223218,0.006494
7,b,-1.358966,0.851101


The group weighted average by category would then be:

In [112]:
grouped = df.groupby('category')
get_wavg = lambda g: np.average(g['data'], weights=g['weights'])
grouped.apply(get_wavg)

category
a    0.302038
b   -0.789039
dtype: float64

In [118]:
#STOCKS

import datetime
def parser(x):
    return datetime.datetime.strptime(x,'%Y-%m-%d')


#3 stocks
###3 Correlated Stocks
url1 = 'https://raw.githubusercontent.com/berserkhmdvhb/stockdata-public/master/GS.csv'
url2 = 'https://raw.githubusercontent.com/berserkhmdvhb/stockdata-public/master/JPM.csv'
url3 = 'https://raw.githubusercontent.com/berserkhmdvhb/stockdata-public/master/MS.csv'

###GS
gs = pd.read_csv(url1,engine='python', header=0, parse_dates=[0], date_parser=parser)
df_gs = gs
#df_gs = df_gs.drop(["Open", "High", "Low", "Close", "Volume"], axis=1)
df_gs.drop(df_gs.columns[[1,2,3,4,6]], axis=1,inplace=True)
df_gs.columns = ['Date','GS']


###JPM
jpm = pd.read_csv(url2,engine='python', header=0, parse_dates=[0], date_parser=parser)
df_jpm = jpm
#df_jpm = df_jpm.drop(["Open", "High", "Low", "Close", "Volume"], axis=1)
df_jpm.drop(df_jpm.columns[[1,2,3,4,6]], axis=1,inplace=True)
df_jpm.columns = ['Date','JPM']


###MS
ms = pd.read_csv(url3,engine='python', header=0, parse_dates=[0], date_parser=parser)
df_ms = ms
#df_ms = df_ms.drop(["Open", "High", "Low", "Close", "Volume"], axis=1)
df_ms.drop(df_ms.columns[[1,2,3,4,6]], axis=1,inplace=True)
df_ms.columns = ['Date','MS']

##Fill Missing Values
def fill_miss(dataframe,ind):
  dataset = dataframe.copy()
  r = pd.date_range(start=dataset.Date.min(), end=dataset.Date.max())
  dataset = dataset.set_index('Date').reindex(r).fillna(dataset.iloc[:, ind].interpolate()).rename_axis('Date').reset_index()
  dataset.iloc[:, ind] = dataset.iloc[:, ind].fillna(dataset.iloc[:, ind].interpolate()) 
  return dataset

df_gs = fill_miss(df_gs,1)
df_jpm = fill_miss(df_jpm,1)
df_ms = fill_miss(df_ms,1)

#change data-time format
df_gs.Date = pd.to_datetime(df_gs.Date)
df_jpm.Date = pd.to_datetime(df_jpm.Date)
df_ms.Date = pd.to_datetime(df_ms.Date)

#result = pd.merge([df_gs, df_jpm, df_ms])
df = pd.merge(df_gs,df_jpm, how='outer', on='Date')
df = pd.merge(df,df_ms,how='outer', on='Date')

df = fill_miss(df,[1,2,3])
df.JPM[0] = df.JPM[1]

In [119]:
df.set_index('Date', inplace=True)
df.head()

,GS,JPM,MS
Date,,,
1999-05-11,55.668247,29.369160,28.974047
1999-05-12,57.934387,29.369160,30.698467
1999-05-13,57.688076,29.971283,31.039766
1999-05-14,55.323395,28.209566,28.776459
1999-05-15,54.912866,28.157530,28.770469


One task of interest might be to compute a DataFrame consisting of the yearly correlations of daily returns (computed from percent changes) with GS . As one way to do
this, we first create a function that computes the pairwise correlation of each column
with the 'GS' column:

In [123]:
gs_corr = lambda x: x.corrwith(x['GS'])


Next, we compute percent change on df using pct_change :

In [126]:
rets = df.pct_change().dropna()
rets.head()

,GS,JPM,MS
Date,,,
1999-05-12,0.040708,0.000000,0.059516
1999-05-13,-0.004252,0.020502,0.011118
1999-05-14,-0.040991,-0.058780,-0.072916
1999-05-15,-0.007421,-0.001845,-0.000208
1999-05-16,-0.007476,-0.001848,-0.000208


Lastly, we group these percent changes by year, which can be extracted from each row
label with a one-line function that returns the year attribute of each datetime label:

In [128]:
get_year = lambda x: x.year
by_year = rets.groupby(get_year)
by_year.apply(gs_corr)

,GS,JPM,MS
1999,1.0,0.657647,0.827030
2000,1.0,0.483563,0.786523
2001,1.0,0.673591,0.844719
2002,1.0,0.731696,0.867641
2003,1.0,0.744768,0.787552
2004,1.0,0.611245,0.747970
2005,1.0,0.542062,0.620669
2006,1.0,0.598647,0.766509
2007,1.0,0.764773,0.806060
2008,1.0,0.637665,0.804358


You could also compute inter-column correlations. Here we compute the annual cor‐
relation between JPM and MS:


In [130]:
by_year.apply(lambda g: g['JPM'].corr(g['MS']))

1999    0.722082
2000    0.582715
2001    0.709776
2002    0.734234
2003    0.777579
2004    0.589471
2005    0.422579
2006    0.628467
2007    0.784261
2008    0.585209
2009    0.751071
2010    0.775610
2011    0.889999
2012    0.734253
2013    0.626871
2014    0.729448
2015    0.869745
2016    0.881089
2017    0.820566
2018    0.879544
dtype: float64

## Example: Group-Wise Linear Regression

executing an ordinary least squares (OLS) regression on each chunk of data:

In [132]:
import statsmodels.api as sm
def regress(data, yvar, xvars):
    Y = data[yvar]
    X = data[xvars]
    X['intercept'] = 1.
    result = sm.OLS(Y, X).fit()
    return result.params

Now, to run a yearly linear regression of JPM on GS returns, execute:

In [133]:
by_year.apply(regress, 'JPM', ['GS'])

,GS,intercept
1999,0.580453,-0.000856
2000,0.376602,-0.000326
2001,0.649000,-0.000223
2002,1.121388,0.000106
2003,0.836668,0.000457
2004,0.525922,0.000179
2005,0.402675,-0.000072
2006,0.442696,0.000075
2007,0.595928,-0.000326
2008,0.720112,0.001374


# 4.Pivot Tables and Cross-Tabulation

A pivot table is a data summarization tool frequently found in spreadsheet programs
and other data analysis software. It aggregates a table of data by one or more keys,
arranging the data in a rectangle with some of the group keys along the rows and
some along the columns.

Pivot tables in Python with pandas are made possible
through the groupby facility described in this chapter combined with reshape operations utilizing hierarchical indexing. DataFrame has a pivot_table method, and
there is also a top-level pandas.pivot_table function.

suppose you wanted to compute a table of group
means (the default pivot_table aggregation type) arranged by day and smoker on
the rows:

In [142]:
tips.shape

(244, 8)

In [144]:
tips.pivot_table(index=['day', 'smoker'])

size       tip   tip_pct  total_bill
day  smoker                                          
Fri  No      2.250000  2.812500  0.151650   18.420000
     Yes     2.066667  2.714000  0.174783   16.813333
Sat  No      2.555556  3.102889  0.158048   19.661778
     Yes     2.476190  2.875476  0.147906   21.276667
Sun  No      2.929825  3.167895  0.160113   20.506667
     Yes     2.578947  3.516842  0.187250   24.120000
Thur No      2.488889  2.673778  0.160298   17.113111
     Yes     2.352941  3.030000  0.163863   19.190588

This could have been produced with groupby directly. Now, suppose we want to
aggregate only tip_pct and size , and additionally group by time . I’ll put smoker in
the table columns and day in the rows:

In [138]:
tips.pivot_table(['tip_pct', 'size'], index=['time', 'day'],
                 columns='smoker')

size             tip_pct          
smoker             No       Yes        No       Yes
time   day                                         
Dinner Fri   2.000000  2.222222  0.139622  0.165347
       Sat   2.555556  2.476190  0.158048  0.147906
       Sun   2.929825  2.578947  0.160113  0.187250
       Thur  2.000000       NaN  0.159744       NaN
Lunch  Fri   3.000000  1.833333  0.187735  0.188937
       Thur  2.500000  2.352941  0.160311  0.163863

We could augment this table to include partial totals by passing margins=True . This
has the effect of adding All row and column labels, with corresponding values being
the group statistics for all the data within a single tier:
    

In [139]:
tips.pivot_table(['tip_pct', 'size'], index=['time', 'day'], 
                 columns='smoker', margins=True)

size                       tip_pct                    
smoker             No       Yes       All        No       Yes       All
time   day                                                             
Dinner Fri   2.000000  2.222222  2.166667  0.139622  0.165347  0.158916
       Sat   2.555556  2.476190  2.517241  0.158048  0.147906  0.153152
       Sun   2.929825  2.578947  2.842105  0.160113  0.187250  0.166897
       Thur  2.000000       NaN  2.000000  0.159744       NaN  0.159744
Lunch  Fri   3.000000  1.833333  2.000000  0.187735  0.188937  0.188765
       Thur  2.500000  2.352941  2.459016  0.160311  0.163863  0.161301
All          2.668874  2.408602  2.569672  0.159328  0.163196  0.160803

Here, the All values are means without taking into account smoker versus non-
smoker (the All columns) or any of the two levels of grouping on the rows (the All
row).

To use a different aggregation function, pass it to aggfunc . For example, 'count' or
len will give you a cross-tabulation (count or frequency) of group sizes:

In [140]:
tips.pivot_table('tip_pct', index=['time', 'smoker'], columns='day',
                 aggfunc=len, margins=True)

day             Fri   Sat   Sun  Thur    All
time   smoker                               
Dinner No       3.0  45.0  57.0   1.0  106.0
       Yes      9.0  42.0  19.0   NaN   70.0
Lunch  No       1.0   NaN   NaN  44.0   45.0
       Yes      6.0   NaN   NaN  17.0   23.0
All            19.0  87.0  76.0  62.0  244.0

In [145]:
tips.pivot_table('tip_pct', index=['time', 'size', 'smoker'],
                 columns='day', aggfunc='mean', fill_value=0)

day                      Fri       Sat       Sun      Thur
time   size smoker                                        
Dinner 1    No      0.000000  0.137931  0.000000  0.000000
            Yes     0.000000  0.325733  0.000000  0.000000
       2    No      0.139622  0.162705  0.168859  0.159744
            Yes     0.171297  0.148668  0.207893  0.000000
       3    No      0.000000  0.154661  0.152663  0.000000
            Yes     0.000000  0.144995  0.152660  0.000000
       4    No      0.000000  0.150096  0.148143  0.000000
            Yes     0.117750  0.124515  0.193370  0.000000
       5    No      0.000000  0.000000  0.206928  0.000000
            Yes     0.000000  0.106572  0.065660  0.000000
       6    No      0.000000  0.000000  0.103799  0.000000
Lunch  1    No      0.000000  0.000000  0.000000  0.181728
            Yes     0.223776  0.000000  0.000000  0.000000
       2    No      0.000000  0.000000  0.000000  0.166005
            Yes     0.181969  0.000000  0.000000  0.158843
       3    No      0.187735  0.000000  0.000000  0.084246
            Yes     0.000000  0.000000  0.000000  0.204952
       4    No      0.000000  0.000000  0.000000  0.138919
            Yes     0.000000  0.000000  0.000000  0.155410
       5    No      0.000000  0.000000  0.000000  0.121389
       6    No      0.000000  0.000000  0.000000  0.173706

In [147]:
# A cross-tabulation (or crosstab for short) is a special case of a pivot table that computes group frequencies.